In [1]:
!git checkout HEAD~10

Previous HEAD position was 91c7b91... Variables defined in ExponentialMovingAverage need not to be shared. (#778)
HEAD is now at 07827aa... Wording change


In [2]:
import numpy as np
import os
import tensorflow as tf
import math

In [3]:
tf.__version__

'0.12.1'

In [4]:
!bazel build -c opt //im2txt/...

Loading: 
Loading: 0 packages loaded
INFO: Analysed 17 targets (0 packages loaded).
Building: no action
INFO: Found 17 targets...
Building: no action
[0 / 6] [-----] BazelWorkspaceStatusAction stable-status.txt
INFO: Elapsed time: 0.191s, Critical Path: 0.00s
[1 / 1] no action
INFO: Build completed successfully, 1 total action


In [5]:
import sys
import collections

In [6]:
sys.path.insert(0, 'bazel-bin/im2txt/run_inference.runfiles/im2txt/')

In [7]:
from __future__ import absolute_import
from im2txt import configuration
from im2txt.inference_utils import caption_generator
from im2txt.inference_utils import vocabulary
from im2txt import show_and_tell_model
from im2txt.inference_utils import inference_wrapper_base

In [8]:
tf.logging.set_verbosity(tf.logging.INFO)

In [9]:
checkpoint_path = '/hdd/models/im2txt_2016_10_05.1000000/model.ckpt-1000000'

In [10]:
class InferenceWrapper(inference_wrapper_base.InferenceWrapperBase):
    """Model wrapper class for performing inference with a ShowAndTellModel."""

    def __init__(self):
        super(InferenceWrapper, self).__init__()
        self.fetches = {}
        self.st_custom_getter = None
        self.post_feed_image = lambda: result 
    
    def build_model(self, model_config):
        model = show_and_tell_model.ShowAndTellModel(model_config, mode="inference")
        model.build()
        return model
    
    def feed_image(self, sess, encoded_image):
        self.fetches['initial_state'] = tf.get_default_graph().get_tensor_by_name('lstm/initial_state:0')
        result = sess.run(fetches=self.fetches,
                                 feed_dict={"image_feed:0": encoded_image})
        self.post_feed_image(result)
        return result['initial_state']

    def inference_step(self, sess, input_feed, state_feed):
        softmax_output, state_output = sess.run(
            fetches=["softmax:0", "lstm/state:0"],
            feed_dict={
                "input_feed:0": input_feed,
                "lstm/state_feed:0": state_feed,
            })
        return softmax_output, state_output, None


In [11]:
model = InferenceWrapper()
config = configuration.ModelConfig()

def _create_restore_fn(self, checkpoint_path, saver):
    print "Someting"
st_variables_map = {}

def st_custom_getter(getter, name, shape, *args, **kwargs):
    if name in st_variables_map:
        return st_variables_map[name]
    print('Restoring', name, shape)
    kwargs['trainable'] = False
    c = None
    name = name_map.get(name) or name
    if reader.has_tensor(name):
        c = reader.get_tensor(name)
    elif reader.has_tensor('InceptionV3/' + name):
        c = reader.get_tensor('InceptionV3/' + name)
    assert(c is not None)
    
    kwargs['initializer'] = tf.constant_initializer(c)

    
    v = getter(name, shape, *args, **kwargs)
    st_variables_map[name]= v
    return v

st_variables = list(st_variables_map.values())

# model.st_custom_getter = st_custom_getter

restore_fn = model.build_graph_from_config(config, checkpoint_path)
tf.trainable_variables()

INFO:tensorflow:Building model.
Instructions for updating:
Please switch to tf.summary.image. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, the max_images argument was renamed to max_outputs.
Instructions for updating:
Please switch to tf.summary.image. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, the max_images argument was renamed to max_outputs.
Instructions for updating:
Please switch to tf.summary.image. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, the max_images argument was renamed to max_outputs.


In [12]:
assert(len(tf.trainable_variables()) == 0)

AssertionError: 

In [13]:
chekpointvars = list(reader.get_variable_to_shape_map().keys())
set(st_variables).difference(set(chekpointvars)), set(chekpointvars).difference(set(st_variables)) 

NameError: name 'reader' is not defined

In [14]:
def test_uninitialized(sess):
    global_vars          = tf.global_variables()
    is_not_initialized   = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]
    if not_initialized_vars:
        print ('Not initalized varibles are: ', [str(i.name) for i in not_initialized_vars])
        # from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
        # print_tensors_in_checkpoint_file(checkpoint_path, all_tensors=True, tensor_name='')
    assert not not_initialized_vars

In [15]:
with tf.Session() as sess:
    restore_fn(sess)
    test_uninitialized(sess)
    const_graph = tf.graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), [
        'lstm/initial_state',
        "softmax",
        "lstm/state"
    ])

INFO:tensorflow:Loading model from checkpoint: /hdd/models/im2txt_2016_10_05.1000000/model.ckpt-1000000
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-1000000
INFO:tensorflow:Froze 766 variables.
Converted 382 variables to const ops.


In [16]:
output_graph = os.path.join(os.path.dirname(checkpoint_path), 'const_' + os.path.basename(checkpoint_path) + '.pb')
with tf.gfile.GFile(output_graph, "wb") as f:
    f.write(const_graph.SerializeToString())

In [17]:
!ls -alh {output_graph} 

-rw-rw-r-- 1 aghasy aghasy 143M Sep 11 21:59 /hdd/models/im2txt_2016_10_05.1000000/const_model.ckpt-1000000.pb


In [20]:
writer = tf.summary.FileWriter(os.path.join(os.path.dirname(checkpoint_path),  'const_' + os.path.basename(checkpoint_path) + '.logdir'), graph_def=const_graph)
writer.flush()